<a href="https://colab.research.google.com/github/Hilal-Urun/tensorflow_everyday/blob/main/Tf_day4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook we are going to classifiy movie ratigs as positive and negative like we did tf_day3, like before it is again a binary classification problem which is very comman in machine learning area but the difference is we are going to examine transfer learning with tensorflow and keras in this time. Furhermore, we are going to use IMDB dataset which contains 50000 reviews from the internet and for this dataset we have equal number of train and test dataset and also contains equal number of positive and negative data.
As we mentioned before, in this notebook we are going to use transfer learning and for this purpose we are going to use tensorflow_hub for loading already trained models.

In [9]:
import os 
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_hub as hub

In [10]:
#now we are going to load our dataset from tensorfow_datasets and split it as training, validationn and test data 
train_data, validation_data, test_data = tfds.load(
    name="imdb_reviews", 
    split=('train[:60%]', 'train[60%:]', 'test'),
    as_supervised=True)

In [11]:
#now we can explore our data and its format then lets just print first 5 examples of our data and their labels 
#0 means negative example and 1 means positve example 
train_examples_batch, train_labels_batch = next(iter(train_data.batch(5)))

In [12]:
train_examples_batch

<tf.Tensor: shape=(5,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell a

In [13]:
train_labels_batch

<tf.Tensor: shape=(5,), dtype=int64, numpy=array([0, 0, 0, 1, 1])>

In [14]:
#time to build our model, and our data consists of sentences, labels are 0 or 1.
#one way to represent the sentences is convert it into embedding vectors, then we will use in our first layers as pretrained text embeddings
#this time we will use pretrained text embeddings frım tensorflow hub but the others also possible 
embedding = "https://tfhub.dev/google/nnlm-en-dim50/2"
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:3])


<tf.Tensor: shape=(3, 50), dtype=float32, numpy=
array([[ 0.5423195 , -0.0119017 ,  0.06337538,  0.06862972, -0.16776837,
        -0.10581174,  0.16865303, -0.04998824, -0.31148055,  0.07910346,
         0.15442263,  0.01488662,  0.03930153,  0.19772711, -0.12215476,
        -0.04120981, -0.2704109 , -0.21922152,  0.26517662, -0.80739075,
         0.25833532, -0.3100421 ,  0.28683215,  0.1943387 , -0.29036492,
         0.03862849, -0.7844411 , -0.0479324 ,  0.4110299 , -0.36388892,
        -0.58034706,  0.30269456,  0.3630897 , -0.15227164, -0.44391504,
         0.19462997,  0.19528408,  0.05666234,  0.2890704 , -0.28468323,
        -0.00531206,  0.0571938 , -0.3201318 , -0.04418665, -0.08550783,
        -0.55847436, -0.23336391, -0.20782952, -0.03543064, -0.17533456],
       [ 0.56338924, -0.12339553, -0.10862679,  0.7753425 , -0.07667089,
        -0.15752277,  0.01872335, -0.08169781, -0.3521876 ,  0.4637341 ,
        -0.08492756,  0.07166859, -0.00670817,  0.12686075, -0.19326553,
 

In [15]:
#our full model follows
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 50)                48190600  
                                                                 
 dense (Dense)               (None, 16)                816       
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 48,191,433
Trainable params: 48,191,433
Non-trainable params: 0
_________________________________________________________________


TensorFlow Hub layer uses a pre-trained model to map a sentence into its embedding vector. The pre-trained text embedding model that we are using splits the sentence into tokens, embeds each token and then combines the embedding. This fixed-length output vector is piped through a fully-connected layer with 16 hidden units.
The last layer is densely connected with a single output node.

In [16]:
#now we will compile our model and it needs to a loss function and optimizer, since our model does binary classification we can use the 
#binary_crossentropy loss function.
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

Now we will train the model for 10 epochs in mini-batches of 512 samples. This is 10 iterations over all samples in the x_train and y_train tensors. While training, we will also track our loss and accuracy

In [17]:
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=10,
                    validation_data=validation_data.batch(512),
                    verbose=1)

Epoch 1/10
30/30 [==============================] - 23s 695ms/step - loss: 0.6232 - accuracy: 0.5844 - val_loss: 0.5627 - val_accuracy: 0.6631
Epoch 2/10
30/30 [==============================] - 21s 685ms/step - loss: 0.4910 - accuracy: 0.7527 - val_loss: 0.4573 - val_accuracy: 0.7740
Epoch 3/10
30/30 [==============================] - 21s 680ms/step - loss: 0.3674 - accuracy: 0.8449 - val_loss: 0.3771 - val_accuracy: 0.8284
Epoch 4/10
30/30 [==============================] - 22s 720ms/step - loss: 0.2693 - accuracy: 0.8985 - val_loss: 0.3328 - val_accuracy: 0.8536
Epoch 5/10
30/30 [==============================] - 23s 739ms/step - loss: 0.1984 - accuracy: 0.9298 - val_loss: 0.3174 - val_accuracy: 0.8683
Epoch 6/10
30/30 [==============================] - 22s 718ms/step - loss: 0.1466 - accuracy: 0.9558 - val_loss: 0.3042 - val_accuracy: 0.8687
Epoch 7/10
30/30 [==============================] - 21s 674ms/step - loss: 0.1073 - accuracy: 0.9714 - val_loss: 0.3097 - val_accuracy: 0.8720

In [18]:
#time to see how the our model performs on test data 
results = model.evaluate(test_data.batch(512), verbose=2)
#this returns two variables which are loss and accuracy 
for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

49/49 - 5s - loss: 0.3595 - accuracy: 0.8557 - 5s/epoch - 103ms/step
loss: 0.359
accuracy: 0.856


This fairly naive approach achieves an accuracy of about 86%. 


That's all for today see you tomorrow, stay with TF🧡